In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import time
import os
import pandas as pd
import gzip

In [2]:
gmvp_precomputed_df = pd.read_csv("gMVP.2021-02-28.csv.gz", compression="gzip", comment="#", sep="\t") # 72x10^6 rows, ~5mins
gmvp_precomputed_df

/tmp/ipykernel_2904277/2258422867.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gmvp_precomputed_df = pd.read_csv("gMVP.2021-02-28.csv.gz", compression="gzip", comment="#", sep="\t") # 72x10^6 rows, 3mins to load


,chrom,pos,ref,alt,gene_symbol,gene_id,transcript_id,protein_position,ref_aa,alt_aa,ref_codon,alt_codon,context,gMVP,gMVP_rankscore
0,1,69091,A,C,OR4F5,ENSG00000186092,ENST00000335137,1,M,L,Atg,Ctg,TAT,0.011946,0.032529
1,1,69091,A,T,OR4F5,ENSG00000186092,ENST00000335137,1,M,L,Atg,Ttg,TAT,0.011946,0.032529
2,1,69091,A,G,OR4F5,ENSG00000186092,ENST00000335137,1,M,V,Atg,Gtg,TAT,0.014359,0.042521
3,1,69092,T,C,OR4F5,ENSG00000186092,ENST00000335137,1,M,T,aTg,aCg,ATG,0.013965,0.040847
4,1,69092,T,G,OR4F5,ENSG00000186092,ENST00000335137,1,M,R,aTg,aGg,ATG,0.020672,0.071386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72616181,Y,25624522,C,A,CDY1,ENSG00000172288,ENST00000306609,554,P,T,Cct,Act,CCC,0.008693,0.020271
72616182,Y,25624522,C,T,CDY1,ENSG00000172288,ENST00000306609,554,P,S,Cct,Tct,CCC,0.010133,0.025510
72616183,Y,25624523,C,G,CDY1,ENSG00000172288,ENST00000306609,554,P,R,cCt,cGt,CCT,0.007198,0.015160
72616184,Y,25624523,C,T,CDY1,ENSG00000172288,ENST00000306609,554,P,L,cCt,cTt,CCT,0.007686,0.016791


In [7]:
small = ""#"_small"
variants_df = pd.read_csv(home_dir+f"models/aa_common/datasets_population_freq/chromosomal_SNVs_with_population_freq{small}.txt", sep="\t")
print(f"Log: #-of total SNVs: {variants_df.shape[0]}")

Log: #-of total SNVs: 5424623


In [8]:
print("Log: merging dbNSFP prediction scores with ground truth population freq data ...")
result_df = gmvp_precomputed_df.merge(variants_df, how="inner", left_on=["chrom", "pos", "ref", "alt"], right_on=["chrom", "pos", "ref", "alt"]) # ~2mins
print(result_df)

Log: merging dbNSFP prediction scores with ground truth population freq data ...
        chrom        pos ref alt gene_symbol          gene_id  \
0           1    3069267   C   T      PRDM16  ENSG00000142611   
1           1    3069290   G   A      PRDM16  ENSG00000142611   
2           1    3186130   G   A      PRDM16  ENSG00000142611   
3           1    3186136   G   A      PRDM16  ENSG00000142611   
4           1    3186136   G   T      PRDM16  ENSG00000142611   
...       ...        ...  ..  ..         ...              ...   
4928615     9  137434055   G   C       NOXA1  ENSG00000188747   
4928616     9  137434055   G   T       NOXA1  ENSG00000188747   
4928617     9  137434059   C   T       NOXA1  ENSG00000188747   
4928618     9  137434067   G   A       NOXA1  ENSG00000188747   
4928619     9  137434067   G   C       NOXA1  ENSG00000188747   

           transcript_id  protein_position ref_aa alt_aa  ... alt_codon  \
0        ENST00000270722                 3      S      F  ...  

In [9]:
from scipy.stats import spearmanr, pearsonr
print(spearmanr(result_df["gMVP"], result_df["mt_freq"]))
print(pearsonr(result_df["gMVP"], result_df["mt_freq"]))

SignificanceResult(statistic=-0.0314947133635774, pvalue=0.0)
PearsonRResult(statistic=-0.03221107939832203, pvalue=0.0)
